# CDP Connector with lazy frames using Polars

## Setup

In [2]:
import pandas as pd
df_main =  pd.read_csv('ACC_minute_data_with_indicators.csv')


## Connect to CDP source using core and refresh token

In [41]:
from salesforcecdpconnector.connection import SalesforceCDPConnection



cur = conn.cursor()



### Get Dataframes

#### Pandas data frame

In [40]:
data_frame = conn.get_pandas_dataframe('SELECT bottles_sold__c, category_name__c, cdp_sys_SourceVersion__c, DataSource__c, DataSourceObject__c, id__c, InternalOrganization__c, item_description__c, item_number__c, store_number__c FROM LiquorSales__dll LIMIT 10000000')

data_frame

Error: Token Renewal failed with code 400

#### Polars lazy frame

## Merging csv to master file

In [2]:
import os
import pandas as pd
df_main = pd.read_csv('ACC_minute_data_with_indicators.csv')
df_main['row_number'] = df_main.reset_index().index
for subdir, dirs, files in os.walk('./'):
    for file in files:
      if (file == 'ACC_minute_data_with_indicators.csv' or file == 'master_file.csv' or file == 'master_file_small.csv'):
        continue

      res=file.split('.')
      if (res[1] == 'csv'):
        print(file + ' ... merging ...')
        df_cur = pd.read_csv(file)
        df_cur['row_number'] = df_cur.reset_index().index
        df_main = df_main.append(df_cur)
        print('current_size = ' + str(df_main.shape[0]))

df_main.to_csv('master_file_extreme_small.csv')


ADANIGREEN_minute_data_with_indicators.csv ... merging ...
current_size = 1031105
ADANIPORTS_minute_data_with_indicators.csv ... merging ...
current_size = 1702138
ADANIENT_minute_data_with_indicators.csv ... merging ...
current_size = 2362614
APOLLOHOSP_minute_data_with_indicators.csv ... merging ...
current_size = 3033648
AMBUJACEM_minute_data_with_indicators.csv ... merging ...
current_size = 3699434


## Testing performance of grouping

In [28]:
df_main = pd.read_csv('master_file_small.csv')

In [43]:

filtered_n = df_main[df_main['volume'] >= 200]
groups_n = filtered_n.groupby(['date']).agg({'high':'mean', 'low':'mean', 'close':'mean'})
groups_n
#groups_n = df_main.groupby(['date'])['high'].agg('mean')
#df_main.shape[0]

,high,low,close
date,,,
2015-02-02 10:18:00+05:30,457.215714,456.770000,456.922857
2015-02-02 10:19:00+05:30,542.683846,542.114615,542.337692
2015-02-02 10:20:00+05:30,373.343077,372.725385,372.961538
2015-02-02 10:21:00+05:30,444.650769,444.045385,444.215385
2015-02-02 10:22:00+05:30,552.745000,552.080000,552.565714
...,...,...,...
2022-10-25 12:21:00+05:30,3697.150000,3694.050000,3695.900000
2022-10-25 12:22:00+05:30,3694.100000,3690.100000,3692.400000
2022-10-25 12:23:00+05:30,3695.300000,3691.250000,3694.950000


In [29]:
#df_main.apply(lambda row: row[3]-row[2], axis=1)
df_main['result2'] = df_main['high'] + df_main['low']

In [31]:
import polars as pl
df_polars = pl.from_pandas(df_main)

In [ ]:

#%timeit df_polars['result2'] - df_polars.apply(lambda row: row[3]+row[2])
df_polars.with_columns([
    (pl.col('high') - pl.col('low')).alias('result')
])

In [41]:
q = (
    df_polars
    .lazy()
    .filter(pl.col('volume') >= 200)
    .groupby(by='date')
    .agg(pl.col('high').mean().alias('high(mean)'))
)
q.collect()

date,high(mean)
str,f64
"""2015-11-17 10:...",497.207692
"""2016-04-06 13:...",536.6225
"""2017-04-26 10:...",805.669412
"""2017-10-23 15:...",822.840625
"""2018-02-22 09:...",780.94375
"""2018-12-05 09:...",1792.8325
"""2021-03-02 11:...",1375.45
"""2022-09-05 15:...",1961.405263
"""2015-02-25 14:...",556.602143


In [18]:
df_polars.apply(lambda row: row[3]-row[2])

apply
f64
1.0
0.3
0.7
0.7
0.05
1.1
0.65
0.0
2.4


In [8]:
import pandas as pd
test_df = pd.read_csv('testFile.csv')
test_df

,index,cat,open
0,0,1,1568128984
1,1,0,1805981289
2,2,4,1196748112
3,3,3,98688856
4,4,4,2060991770
...,...,...,...
599999995,599999995,0,1796254434
599999996,599999996,4,1161780732
599999997,599999997,3,1933478526
599999998,599999998,4,1866351219


In [14]:
test_filter = test_df[test_df['open'] >= 4000000]
test_group = test_filter.groupby(['cat']).agg({'open':'mean'})
test_group

,open
cat,
0,1.075742e+09
1,1.075804e+09
2,1.075440e+09
3,1.075664e+09
4,1.075678e+09


In [10]:
import polars as pl
test_polars = pl.from_pandas(test_df)

In [13]:
q = (
    test_polars
    .lazy()
    .filter(pl.col('open') >= 4000000)
    .groupby(by='cat')
    .agg(pl.col('open').mean().alias('open(mean)'))
)
q.collect()

cat,open(mean)
i64,f64
0,1.0757e9
2,1.0754e9
4,1.0757e9
1,1.0758e9
3,1.0757e9
